In [5]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("beyondb_authors_reply_with_emotions.csv")

# Remove the column
if "cleaned_authors_comment" in df.columns:
    df = df.drop(columns=["cleaned_authors_comment"])

# Remove rows where Author_replied == 0
df = df[df["Author_replied"] != 0]

# Save new file
df.to_csv("a.csv", index=False)

print("File saved as a.csv")


File saved as a.csv


In [8]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("unlabeled_with_author_emotion_predictions.csv")

# Remove the column
if "cleaned_post_content" in df.columns:
    df = df.drop(columns=["cleaned_post_content"])
if "cleaned_authors_comment" in df.columns:
    df = df.drop(columns=["cleaned_authors_comment"])
if "cleaned_comments_content" in df.columns:
    df = df.drop(columns=["cleaned_comments_content"])

if "comments_sentiment" in df.columns:
    df = df.drop(columns=["comments_sentiment"])
if "cleaned_authors_comment" in df.columns:
    df = df.drop(columns=["cleaned_authors_comment"])
if "input_text" in df.columns:
    df = df.drop(columns=["input_text"])    


# Remove rows where Author_replied == 0
df = df[df["Author_replied"] == 0]

# Save new file
df.to_csv("b.csv", index=False)

print("File saved as a.csv")

File saved as a.csv


In [ ]:
import pandas as pd
import ast

# Load data
df = pd.read_csv("a.csv")

# Convert dict-like strings into actual dicts if needed
df["pred_authors_reply_probabilities"] = df["pred_authors_reply_probabilities"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Extract probability that matches the emotion label
df["matched_probability"] = df.apply(
    lambda row: row["pred_authors_reply_probabilities"].get(row["pred_authors_reply_emotions"], None),
    axis=1
)

df = df.drop(columns=["pred_authors_reply_probabilities"])

# Save output
df.to_csv("a.csv", index=False)
print("Saved as a.csv")


Saved as a.csv


In [11]:
import pandas as pd

# Load both CSVs
df1 = pd.read_csv("a.csv")
df2 = pd.read_csv("b.csv")

# Merge by stacking rows
merged = pd.concat([df1, df2], ignore_index=True)

# Save merged file
merged.to_csv("merged_author_A_B_final_emotions.csv", index=False)

print("Merged file saved as merged_author_A_B_final_emotions.csv")


Merged file saved as merged_author_A_B_final_emotions.csv


In [12]:
import pandas as pd

df = pd.read_csv("merged_author_A_B_final_emotions.csv")
print("Total rows:", len(df))


Total rows: 12763


In [15]:
import pandas as pd

# Load both CSVs
posts = pd.read_csv("beyondb_post_predictions.csv")  # forum_name,thread_id,cleaned_post_content,pred_emotions,pred_probabilities
authors = pd.read_csv("merged_author_A_B_final_emotions.csv")  # forum_name,thread_id,Author_replied,pred_authors_reply_emotions,pred_authors_reply_probabilities

# Merge on forum_name and thread_id
merged = pd.merge(
    posts,
    authors,
    on=["forum_name", "thread_id"],
    how="inner"   # use "inner" so only matching thread_id/forum_name pairs are kept
)

# Map Author_replied -> Cohort (1 -> A, 0 -> B)
def to_cohort(x):
    try:
        v = int(x)
    except Exception:
        return None
    if v == 1:
        return "A"
    elif v == 0:
        return "B"
    else:
        return None

merged["Cohort"] = merged["Author_replied"].apply(to_cohort)

# Rename columns as requested
merged = merged.rename(columns={
    "cleaned_post_content": "post_content",
    "pred_emotions": "initial_emotion",
    "pred_probabilities": "initial_prob",
    "pred_authors_reply_emotions": "final_pred_emotion",
    "pred_authors_reply_probabilities": "final_pred_prob"
})

# Select columns in the final desired order
final_df = merged[
    [
        "forum_name",
        "thread_id",
        "post_content",
        "Cohort",
        "initial_emotion",
        "initial_prob",
        "final_pred_emotion",
        "final_pred_prob"
    ]
]

# Save to new CSV
final_df.to_csv("all_threads_with_emotions.csv", index=False)
print("Saved as all_threads_with_emotions.csv")


Saved as all_threads_with_emotions.csv


In [18]:
import ast
import numpy as np

# Convert dict-like strings into dicts, leave floats/NaN unchanged
final_df["initial_prob"] = final_df["initial_prob"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith("{") else x
)

# Extract matching probability ONLY when it's a dict
final_df["initial_prob"] = final_df.apply(
    lambda row: row["initial_prob"].get(row["initial_emotion"], None)
    if isinstance(row["initial_prob"], dict)
    else row["initial_prob"],
    axis=1
)

# Save updated dataframe to CSV
final_df.to_csv("final_dataset.csv", index=False)
print("Saved as final_dataset.csv")


Saved as final_dataset.csv


/var/folders/v8/hnghsvb16hqgpbjwl4mg79rw0000gn/T/ipykernel_7378/1902560717.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["initial_prob"] = final_df["initial_prob"].apply(
/var/folders/v8/hnghsvb16hqgpbjwl4mg79rw0000gn/T/ipykernel_7378/1902560717.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["initial_prob"] = final_df.apply(
